# IRISpy Demo for IRIS-9 2018

This notebook outlines the current functionality of IRISpy for the IRIS-9 workshop. To execute code cells, press SHIFT + ENTER.

__N.B. You must have IRISpy installed to work with this tutorial.  For install instructions, see http://docs.sunpy.org/projects/irispy/en/latest/installation.html __

__During the IRIS-9 Workshop (2018-06-25), this notebook was run with the Development Dependendcies Install (http://docs.sunpy.org/projects/irispy/en/latest/installation.html#development-dependencies-install ).  Runnin it with the Stable Dependencies Install should still be valid but you may encounter slightly different behaviour.__

__WARNING__: *The IRISpy is still under heavy development and could change at any time.  However, we are striving towards a stable release.*

To get involved, check out http://docs.sunpy.org/projects/irispy/en/latest/

In [ ]:
# Import some packages to help us display plots and handle physical units
import matplotlib.pyplot as plt
import astropy.units as u

In [ ]:
# Set matplotlib setting so that plots are displayed as active windows within notebook.
# Be sure to close the active window by clicking the blue button at the top right of the plot window 
# before trying to plot a new window.
% matplotlib notebook

## Spectrograph Data

### Reading in FITS files.

IRISpy provides easy-to-use reader functions to read in SJI and spectrograph files.  Simply supply a filename or ist of filenames.

In [ ]:
from irispy.spectrograph import read_iris_spectrograph_level2_fits

In [ ]:
# Define spectrograph filenames by entering strings inside the empty list below.
sg_files = ["iris_l2_20180617_032355_3690215148_raster_t000_r00000.fits",
            "iris_l2_20180617_032355_3690215148_raster_t000_r00001.fits"]

In [ ]:
# Read in spectrograph data from FITS files to an IRISSpectrograph object.
my_sg = read_iris_spectrograph_level2_fits(sg_files)

### Structure of the Spectrograph Data Classes

In [ ]:
my_sg

A summary of the spectral windows in the ```IRISSpectrograph``` object can be obtained from the ```spectral_windows``` property.

In [ ]:
my_sg.spectral_windows

Data from different spectral windows is contained in the ```.data``` attribute, which is indexed by the spectral window name.  Let's focus on the ```C II``` window.

In [ ]:
my_cii = my_sg.data["C II 1336"]

In [ ]:
my_cii

The ```IRISSpectrogramCubeSequence``` is a sequence of data cubes, each representing one raster scan.  Each raster cube is represented by an ```IRISSpectrogramCube```.  Let's extract the 0th scan.

In [ ]:
my_cii_scan0 = my_cii[0]

In [ ]:
my_cii_scan0

Each ```IRISSpectrogramCube``` combines __data__, __WCS tranformations__, __uncertainties__ (calculated on intialisation), data __unit__ and data __mask__ (identifying bad or non-exposed pixels), __metadata__ and __auxiliary data__ (e.g. measurement times, exposure times).

In [ ]:
my_cii_scan0.data 

# Note that that pixels with -200 DN are not exposed.

In [ ]:
my_cii_scan0.uncertainty

# Uncertainties are derived from counting statistics and readout noise of relevant detector.
# Unexposed pixels will not have valid uncertainties.

In [ ]:
my_cii_scan0.mask # True means pixel IS masked, i.e. data is not good.

__N.B.__ Note that here, the value of the mask is True if the data is masked, i.e. the data is bad.

In [ ]:
my_cii_scan0.unit

# This is an astropy unit.  See further down this notebook for more discussion of astropy units and quantities.

In [ ]:
# The WCS object contains all the information on the transformations between pixel and real world coordinates.
my_cii_scan0.wcs

In [ ]:
# Metadata cn be found in the meta attribute.
my_cii_scan0.meta

Auxiliary data, like measurement times, exposure times, etc., are held in the extra_coords property.  The data are stored as entries in a dictionary.  To see the types of auxiliary data included, use the standard ```keys``` methods of Python dictionaries.

In [ ]:
my_cii_scan0.extra_coords.keys()

Each entry, e.g. ```'exposure time'``` is itself a dictionary.

In [ ]:
my_cii_scan0.extra_coords["exposure time"]

The first key in the exposure time dictionary is ```axis``` which gives the data axis of ```my_cii_scan0``` to which exposure time corresponds. Here were can see it is zero. The second key is ```value``` which gives the exposure time value at each pixel along the axis. So we can see that the number of exposure times equals the length of the 0th axis.

In [ ]:
print("Number of exposure times = {0}\nLength of axis = {1}".format(
    len(my_cii_scan0.extra_coords["exposure time"]["value"]),
    my_cii_scan0.dimensions[my_cii_scan0.extra_coords["exposure time"]["axis"]]))

### IRIS Data Manipulation

#### Exposure Time Correction

We can apply exposure time corrections and convert the units of our data between DN, photons and radiance with ease in IRISpy.

If you want to see how this is done, just look at the source code either on GitHub or on your local machine.  __IRISpy is open-source!__

#### Apply exposure time correction

In [ ]:
my_cii[0].unit

In [ ]:
my_cii.apply_exposure_time_correction()

In [ ]:
my_cii[0].unit

Note how the unit is now in DN/s.  The values in the data and uncertainty attributes have been scaled accordingly.  Check out at ```my_sg.data``` nd ```my_sg.uncertainty``` to verify.

To undo the exposure time correction, call ```apply_exposure_time_correction``` setting the ```undo``` keyword to ```True```.

In [ ]:
my_cii.apply_exposure_time_correction(undo=True)

In [ ]:
my_cii[0].unit

Note that unit is now back in DN.

#### Convert Data Betweeen DN, Photons, Radiance

To convert your data and uncertainty values between DN, photons, and radiance, use the ```convert_to``` method.  To track the effect of this method, we will again look how the unit changes.  But again, the data and uncertainty values are altered accordindly.

In [ ]:
my_cii[0].unit

We start in units of DN.  To convert to photons, call the ```convert_to``` method with the string ```"photons"``` as the argument.

In [ ]:
my_cii.convert_to("photons")

In [ ]:
my_cii[0].unit

Note that the unit is now in photons.  To convert the data, uncertainties and unit to radiance, enter ```"radiance"``` as the argument to the ```convert_to``` method.

In [ ]:
my_cii.convert_to("radiance")

In [ ]:
my_cii[0].unit

Now the unit is in radiance units.  To get back to DN, simply put ```"DN"``` into the ```concert_to``` method.

In [ ]:
my_cii.convert_to("DN")

In [ ]:
my_cii[0].unit

In this case, the unit is ```DN/s```.  This is because the exposure time correction and the unit conversion are kept separate.  The one except is the conversion to radiance, which requires the exposure time correction.  Therefore, when we convert from radiance to DN, the inverse time due to the exposure time is maintained.  To undo this and get back to simple data numbers, undo the exposure time correction as we did above.

In [ ]:
my_cii.apply_exposure_time_correction(undo=True)

In [ ]:
my_cii[0].unit

### Plotting

Let's produce a quicklook animation of our spectral window data.

In [ ]:
my_cii.plot()

By default, the y-axis corresponds to position along the slit and the x-axis to wavelength.  If the data is 4D, the bottom slider corresponds to the raster repeat axis and the top slider corresponds to the slit location axis.  Press play on a slider to animate the image.  (__Note that if you are using the stable dependencies install, the 0ths and 1st axes may have been combined into a single slider.  But the future stable dependencies install will exhibit the behaviour described above.__)

It is possible to customize the plot, e.g. change the plot axes, change the color map, add axis labels, etc. but due the basic scope of this introductory tutorial we will leave that for another day.

### Slicing/Indexing

#### 4D Slicing

Say we have a region of interest within our data between the 100th and 175th pixels along the slit at the 3rd slit position, in the wavelength range around the 75-125 pixels.  To get extract this region of interest is simply a case of slicing the ```IRISSpectrogramCubeSequence``` object.

__N.B. depending on the spectrograph files you've read in, you may need to change these index numbers to be within the ranges of your axes.  Let's start by checking the dimensionality of the IRISSpectrumCubeSequence using the ```dimensions``` property.__

In [ ]:
my_cii.dimensions

In [ ]:
# We can see what physical properties each axis corresponds to by using the world_axis_physical_types property.
# Note the 'meta.obs.sequence' refers to the repeat raster access.
my_cii.world_axis_physical_types

To isolate our region of interest, we can index the ```IRISSpectrogramCubeSequence``` as if it were a simple array.  This way the __data, uncertainties, mask, coordinate transformations, and relevant auxiliary data__ are sliced accordindly.  This reduces the need to perform repetitive tasks that by their repetitive nature, an lead to mistakes.

In [ ]:
cii_roi = my_cii[:, 3, 100:175, 50:100]

In [ ]:
cii_roi.dimensions

In [ ]:
cii_roi.world_axis_physical_types

By looking at the ```dimensions``` and ```world_axis_physical_types```, we can see that the ```IRISSpectrogramCubeSequence``` has lost its longitude axis and the remaining axes have been sliced as requested.

__Exercise:__ Manually check the dimensionality of the data, uncertainty, etc. of the ```IRISSpectrogramCube```s within the ```IRISSpectrogramCubeSequence.data``` attribute to verify their dimensionalities been reduced correctly.

We can also use this slicing to easily plot a 1D spectrum from a single pixel.  Let's plot the pixel from the 0th raster scan, and the 30th pixel along the slit.

__N.B. Again, depending on the files you read in and the region you defined above, you may have to select a different pixel with the range of your data.__

In [ ]:
cii_roi[0, 30].plot()

#### 3D Slicing

Above we treated the data as though it's 4D.  But depending on our science goals and the observing campaign, it may be convenient to think of data as it were 3D, i.e. that the repeat raster and slit location axes are combined so that the exposure are simply ordered in time.  The ```IRISSpectrogramCubeSequence``` object makes this easy by providing ```cube_like``` inspection and slicing.

In [ ]:
my_cii.dimensions

In [ ]:
my_cii.cube_like_dimensions

Note that the data is represented as 3D where the 0th cube-like axis has a length equal to the 0th x 1st axes' lengths.

In [ ]:
my_cii.world_axis_physical_types

In [ ]:
my_cii.cube_like_world_axis_physical_types

Also note that the raster repeat axis has been discarded in favour of just the longitude axis alone.

We can also slice the data as though it's 3D.  Let's say we want the first 35 exposures, regardless of what raster scan and slit location they correspond to.  And also suppose we only want the 100-175 pixels along the slit and only the 50-100 pixels along the wavelength direction.  This sounds like a comples slicing operation.  But we can use the ```index_as_cube``` property to index the ```IRISSpectrogramCubeSequence``` as if it were a simple 3D array.

In [ ]:
cii_roi_3d = my_cii.index_as_cube[0:35, 100:175, 50:100]

Note that the data now have the 3D dimensions we expect.

In [ ]:
cii_roi_3d.cube_like_dimensions

In [ ]:
cii_roi_3d.cube_like_world_axis_physical_types

However, also note that the data can still be represented in 4D.  In this case note that the 1st quantity, corresponding to the slit location axis, now has two values.  The 0th corresponds to the 0th raster scan.  The value is 32 showing that all slit locations are included.  The 1st, corresponding to 1st raster scan, has a value of 3, showing that only the first 3 slit locations of that raster scan were included in our cube-like slicing operation.

In [ ]:
cii_roi_3d.dimensions

In [ ]:
cii_roi_3d.world_axis_physical_types

#### Cropping by Real World Coordinates

(__Currently only works for Development Dependencies Install__)

We may want to identify a region of interest based on real world coordinates, rather than pixel indices.  To do this, we can use the ```IRISSpetrogramCube.crop_by_coords``` method.  Note that this method currently only exists for ```IRISSpetrogramCube``` and not ```IRISSpetrogramCubeSequence```.  So let's try it with ```my_cii_scan0```.  To use, supply the real world coordinates of the lower corner and upper corner of the region of interest.

__(N.B. Depending on the files you read in at the start of this notebook, you may have to alter the values input to this method below.)__

In [ ]:
rwc_cropped_cii_scan0 = my_cii_scan0.crop_by_coords(lower_corner=(-250*u.arcsec, 60*u.arcsec, 1334*u.angstrom), 
                                                    upper_corner=(-220*u.arcsec, 90*u.arcsec, 1335*u.angstrom))

In [ ]:
rwc_cropped_cii_scan0

Note that if the data is rotated relative to the physical axes, ```crop_by_coords``` finds the smallest region aligned with the data that includes all of the region marked out by the real world coordinates supplied.

Also note that this method does not do any interpolation.  Instead it finds the pixel values corresponding to supplied real world coordinates and rounds them up or down to the nearest integer.  So a small change in the input to ```crop_by_coords``` may not affect the output.

### A Quick Tangent on Astropy Units and Quantities

Astropy units are objects representing physical units.  Astropy quantities are numbers or arrays of number which have a unit associated with them.

In [ ]:
import astropy.units as u

Units often have a symbol representation and a name representation.

In [ ]:
u.s

In [ ]:
u.second

To construct a Quantity supply the Quantity contructor with number or array and unit object.

In [ ]:
my_time = u.Quantity([1, 2, 3], unit=u.s)

In [ ]:
my_time

Or multiply a number or list/array or numbers with a unit object.

In [ ]:
my_distance = [20, 50, 80] * u.m

In [ ]:
my_distance

#### Quantity Arithmetic

Quantities can be used in simple arithmetic operations.  Not only are the numbers handled but the units are changed as appropriate.

In [ ]:
my_speed = my_distance / my_time

In [ ]:
my_speed

Quantities check unit compatibility before performing the operation and stop you doing unphysical things.

In [ ]:
my_distance + my_time

#### Unit Convervsion

Astropy Quantities can perform valid unit conversions.  Use the ```to``` method and supply it with the unit object to which you wish to convert.

In [ ]:
my_distance.to("angstrom")

Incompatible unit conversions will raise an error.

In [ ]:
my_distance.to(u.s)

### Coordinate Transformations

To get an array of real world values for every pixel along an axis for a single raster scan, say the wavelength axis (2nd axis using zero-based counting), use the ```axis_world_coordinates``` method.

Note these methods only exist for ```IRISSpectrogramCube```s.  So in this example, let's let's use the 0th raster scan from the original C II spectral window data, stored in ```my_cii_scan0```.

In [ ]:
my_cii_scan0.world_axis_physical_types

In [ ]:
wavelength = my_cii_scan0.axis_world_coords(2)

In [ ]:
wavelength

In [ ]:
wavelength.shape, my_cii_scan0.dimensions[-1]

In [ ]:
type(wavelength)

Note that the output is an astropy quantity with the same length as the wavelength axis.

You can also use a substring of the ```world_axis_physical_types``` label for the axis.

In [ ]:
my_cii_scan0.world_axis_physical_types

In [ ]:
my_cii_scan0.axis_world_coords("wl")

#### Dependent Axes

Note that not all axes are independent, like wavelength in the above example.  Some are dependent on others, for example, latitude and longitude.  Depending on the latitude you're at, the longitude value at certain pixel along the longitude axis will have a different value.

In [ ]:
lon = my_cii_scan0.axis_world_coords("lon")

In [ ]:
lon

In [ ]:
my_cii_scan0.dimensions

In [ ]:
lon.shape

Note how the dimensionality of the longitude quantity is the same as the longitude axis x the latitude axis.  The same would be true if we asked for the latitude.

Multiple axis labels/indices can be entered and multiple arrays are returned.  Entering no arguments causes real world coordinates for all axis to be returned.  In these cases a tuple of quantities is returned.

In [ ]:
all_world_coords = my_cii_scan0.axis_world_coords()

In [ ]:
all_world_coords

#### Pixel to World & World to Pixel

If you want to convert the other way, from world coordinates to pixels, or are interested in only a subset of pixels and want a little more efficiency, you can use the ```pixel_to_world``` and ```world_to_pixel``` methods.

Let's convert just the (0, 0, 0) pixel to world coordinates.

In [ ]:
my_cii_scan0.pixel_to_world(u.Quantity([0], unit=u.pix), 
                            u.Quantity([0], unit=u.pix), 
                            u.Quantity([0], unit=u.pix))

Let's now take that output and convert back to pixel coordinates.

In [ ]:
my_cii_scan0.world_to_pixel(0.00229961*u.deg, -0.09773224*u.deg, 1.3319e-07*u.m)

Notice that within numerical error, we can got back to (0, 0, 0) in pixel space.

As many pixels as you like can be entered to the ```pixel_to_world``` and ```world_to_pixel``` methods.  For example, let's also convert the (7, 7, 7) pixel:

In [ ]:
my_cii_scan0.pixel_to_world(u.Quantity([0, 7], unit=u.pix), 
                            u.Quantity([0, 7], unit=u.pix), 
                            u.Quantity([0, 7], unit=u.pix))

## SJI

Let's now explore the tools available for handling SJI data.

### Reading FITS files

In [ ]:
from irispy.sji import read_iris_sji_level2_fits

In [ ]:
# Define a single SJI filename here.
my_sji_file = "iris_l2_20180617_032355_3690215148_SJI_1330_t000.fits"

In [ ]:
# Enter the file name into the SJI FITS reader.
my_sji = read_iris_sji_level2_fits(my_sji_file)

### Structure of the IRISMapCube Class

In this case, we will assume you have entered a single file and so we have produce an ```IRISMapCube```.

In [ ]:
my_sji

Access the data, uncertainties, mask, unit, WCS transformation, metadata and auxiliary data via the following attributes, just like with the ```IRISSpectrogramCube```.

__(The code is uncommented to avoid large print outs to the screen.  Uncomment and run to see the output.)__

In [ ]:
#my_sji.data

In [ ]:
#my_sji.uncertainty

In [ ]:
#my_sji.mask

In [ ]:
#my_sji.unit

In [ ]:
#my_sji.wcs

In [ ]:
#my_sji.meta

In [ ]:
#my_sji.extra_coords

### Inspecting the Data

Analaogous methods and properties to those of the ```IRISSpectrogramCube``` class for inspecting the data also exists for ```IRISMapCube```.

In [ ]:
my_sji

In [ ]:
my_sji.dimensions

In [ ]:
my_sji.world_axis_physical_types

We can produce a movie using the plot method.

In [ ]:
my_sji.plot()

Again, customizing this plot is possible, but not explained here.

### Slicing/Indexing

Say we want have a region of interest within our data between the 10th and 20th frames in time and between the 50th and 150th pixel in the longitude-direction and the 75th and 175th in the latitude-direction.

In [ ]:
sji_roi = my_sji[10:20, 50:150, 75:175]

In [ ]:
sji_roi.dimensions

In [ ]:
sji_roi.world_axis_physical_types

In [ ]:
sji_roi.plot()

### Exposure Time Correction

In [ ]:
sji_roi.unit

In [ ]:
sji_roi = sji_roi.apply_exposure_time_correction()

In [ ]:
sji_roi.unit

In [ ]:
sji_roi = sji_roi.apply_exposure_time_correction(undo=True)

In [ ]:
sji_roi.unit

### Dust Mask

The ```IRISMapCube``` has a method that enables you to mask or unmask pixels which are affected by dust.  This is similar to the dustbuster routine in SSW except that it doesn't interpolate the data in the dusty pixels.  It simply identifies them in the mask.  To mask the dusty pixels, do:

In [ ]:
my_sji.apply_dust_mask()

To unmask dusty pixels, do:

In [ ]:
my_sji.apply_dust_mask(undo=True)